# <mark> <b> > 1. </b> Mapeamento e analise do  Pipeline </mark>

<b>1_map_analise_pipeline_v0.ipynb </b> |  Atual notebook com as funçoes de mapeamento e analise do pipeline

### Modules & config

In [29]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle



# Modulos da solucao
import modules.cronometro as cron
import modules.utils as utl
import modules.trata_pdf as tpdf


In [2]:
#### Config - Mapeamento e Analise do pipeline

# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'

# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"


# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"



#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

### Funcoes para Analise do pipeline

In [16]:
# Unica funcao que devera ser movida para junto das demais funcoes de dicionario
def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
    
    # Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)
#print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

## <mark> <b> 3.0</b>  Mapeamento e Analise do pipeline </mark>

In [30]:
# 1. XXX apagar arquivos zone
utl.apagar_zone(documentos_extracao_path)

In [13]:
# SOMENTE PARA TESTE DE FUNCAO
#fake_parent_document_unique_id = 'f976c128-1f41-4551-bffd-fac687c1c8b2'

# Busca proximo Batch caso nao esteja rodando email
batch_name = utl.busca_proximo_batch(conf_export_plan_path)
batch_name

'Batch_23'

In [31]:
# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos - ESSA E A UNICA FUNCAO QUE ITERA NO DIRETORIO
def scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status):
    
    doc_info = {}
    resumo = {}
    raw_document = []
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            nome_arquivo = file

                
            palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)
            acao_executada = "Analise"
            informations = ' '    
            file_name = file.lower()    
            file_path = os.path.join(root, file)
            if file.lower().endswith('.zip') or file.lower().endswith('.rar') or file.lower().endswith('.7z'):
                compressed_file_path = file_path
                compressed_file_name = os.path.basename(compressed_file_path)
                compressed_file_hash = utils.generate_file_hash(compressed_file_path)
                compressed_file_unique_id = utl.generate_unique_id()
                parent_document_unique_id = compressed_file_unique_id
                level = 2
            new_path_name = os.path.join(output_dir, file)
            if file.lower().endswith('.pdf'):
                doc_one_page, nro_pgs = tpdf.analisa_nro_pages(file_path)
                one_page = doc_one_page
                num_page = nro_pgs
                document_unique_id = utl.generate_unique_id()
                level = 3
            else:
                one_page = False
                num_page = 0    
            # if doc_one_page:
            #     one_page = doc_one_page
            # else:
            #     one_page = False    
            #             rotulo = 'pdf_mul_paginas'
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
                
            #print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    
            
            new_row = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "batch": batch_name,
                "fase_processo": fase,
                "nome_atividade": atividade,
                "status_documento": status,
                "acao_executada": acao_executada,
                "original_file_name": file,
                "directory": folder_name,
                "one_page": one_page,
                "pages": num_page,
                "palavra_chave": palavra_chave,
                "document_tag": rotulo,
                "action_item": acao_sugerida,
                "level": level,
                "document_unique_id": utl.generate_unique_id(),
                "parent_document_unique_id": parent_document_unique_id,
                "file_hash": utl.generate_file_hash(file_path),
                "file_path": file_path,
                "informations": informations,
            }
            raw_document.append(new_row)

            
            # print(f'seq: {i} | file: {file}'
            i += 1
    df_trans_pipe = pd.DataFrame(raw_document)
      
                
    return df_trans_pipe, raw_document

In [38]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'root_analise'

documentos = []

df_root_pipe, documentos = scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status)

In [39]:
# 0. Acertando o Index
df_root_pipe.set_index('document_unique_id', inplace=True)
df_root_pipe

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,pages,palavra_chave,document_tag,action_item,level,parent_document_unique_id,file_hash,file_path,informations
document_unique_id,,,,,,,,,,,,,,,,,,,
6f1a0643-7918-493d-8d93-39ae66a23b5d,1,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,MESQUITA_PDF_31282023_2258.zip,root_dir,False,0,zip,doc_zip,NO_PROCESS,2,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,8d7038d712373364fa4c7680a887a0ceed01c8692d6958...,pipeline_extracao_documentos/2_documentos_para...,
3350d180-8ed9-4334-8189-796f4499d851,2,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,1.pdf,teste,True,1,default,prov_nota_fiscal,PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,66a7db9ee1500d5f9fa5da26563cfd7b68f1f5ba3daba2...,pipeline_extracao_documentos/2_documentos_para...,
93d95698-277c-40b2-b501-cb84476109bb,3,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,Livro de Registro do ISSQN.pdf,115964,False,4,livro,prov_livro_registro,NO_PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,b960962503987f6e05f5646d71a789facfe4e80ccb8890...,pipeline_extracao_documentos/2_documentos_para...,
84bc5464-fcdb-4245-8b10-b2167a80405b,4,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,2023 -5.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,23a28a363c2d2c8b700ac4775164f7c0f0e2d6cef6166d...,pipeline_extracao_documentos/2_documentos_para...,
2752b1a8-9e22-477a-a317-5eb636327c9d,5,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,2023 -7.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,54045f4c09341d9f8d69438e7afe71eff46bb4e731392b...,pipeline_extracao_documentos/2_documentos_para...,
e1ed80a3-d062-4ce6-ade8-70ad8ea746be,6,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,2023 -4.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,ddd09bb806dc79e98a74c5cf1adc6a5bd23ea1b4f1bfa7...,pipeline_extracao_documentos/2_documentos_para...,
7f909b5e-0ee5-48ca-bd4a-c4bc58b94daa,7,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,2023 -6.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,8910a092d3aa53b6a1eb805c783245493760a1c46dadfa...,pipeline_extracao_documentos/2_documentos_para...,
1816b01a-8aa8-4831-9e0b-14c42dfbecde,8,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,2023 -3.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,25387d066a46925acba7ddfe3cd97e2e70f92838ef4312...,pipeline_extracao_documentos/2_documentos_para...,
d9a5adbb-84c5-4df9-93e8-25bb809ed940,9,24/09/2023 09:23:12,Batch_23,analise,scan_analise,root_analise,Analise,2023 -8.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,8c142beb-753b-4ad8-adff-4ca10f0bf7e7,8be881c377bf3064ef61f014fd14602cc736a73b69de8c...,pipeline_extracao_documentos/2_documentos_para...,


### exportando o map_analise_path

In [48]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_root_pipe
df_root_pipe.to_excel(file_path_root_pipe, index=True)

## Pesquisas e outros

### Atualizando o valor da coluna status_documento

In [44]:
def update_status(row):
    if row['seq'] in sequencia:
        row['status_documento'] = 'PREPROCESS_EXTRACT'
    return row

In [45]:
sequencia = [2, 8, 6]

In [46]:
df_root_pipe = df_root_pipe.apply(update_status, axis=1)
df_root_pipe

In [ ]:
# Podemos chegar de forma mais rapida
subset_df.loc[subset_df['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')

In [33]:
# 7. XXX Retiro o indice do DF - Resetando o índice e mantendo o índice original como uma nova coluna
df_analise_pipe.reset_index(inplace=True)


# XXX Criando uma nova coluna no DF
df_conferencia.insert(loc=50, column='original_file_name', value=df_conferencia['file_path'].apply(lambda x: os.path.basename(x)))

df['coluna_cnae'] = df['coluna_cnae'].apply(lambda x: x.strip() if isinstance(x, str) else x)


# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_root_pipe[df_root_pipe['one_page'] == False]



# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')


# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)


# 4. Filtrando Linhas Baseadas em Valores em uma Lista

valores = [11, 16, 30, 41]
subset_df = df_scan_pipe[df_scan_pipe['seq'].isin(valores)]

# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df = df_scan_pipe.iloc[:10]




registro_específico = df_scan_pipe.loc['e1f4b1af-30f3-45d2-85a7-1bb895bd5325']



df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: row['coluna1'] * 2 if row['coluna2'] > 0 else row['coluna1'], axis=1)


# Atualizei o DF baseado em condiçao de outra coluna
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: False if row['status_documento'] == "Template_encontrado" else row['pdf_pesquisavel'], axis=1)


# Podemos chegar de forma mais rapida
df_analise_pipe.loc[df_analise_pipe['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False


# 8. XXX Concatenando os DataFrames
df_analise_pipe = pd.concat([df_analise_pipe, df_docs_splitados], ignore_index=True)

# 9. XXX Volto novamente o indice do DF
df_analise_pipe.set_index('document_unique_id', inplace=True)

NameError: name 'df_conferencia' is not defined